In [16]:
import graphene
import typing

## Some introductions

In [17]:
graphene.ObjectType?

Init signature: graphene.ObjectType(*args, **kwargs)
Docstring:     
Object Type Definition

Almost all of the GraphQL types you define will be object types. Object types
have a name, but most importantly describe their fields.
File:           ~/.local/share/virtualenvs/gpug-9Pz75Evm/lib/python3.6/site-packages/graphene/types/objecttype.py
Type:           SubclassWithMeta_Meta


In [36]:
graphene.Field?

Init signature: graphene.Field(type, args=None, resolver=None, source=None, deprecation_reason=None, name=None, description=None, required=False, _creation_counter=None, default_value=None, **extra_args)
Docstring:      <no docstring>
File:           ~/.local/share/virtualenvs/gpug-9Pz75Evm/lib/python3.6/site-packages/graphene/types/field.py
Type:           type


In [38]:
graphene.Scalar?

Init signature: graphene.Scalar(*args, **kwargs)
Docstring:     
Scalar Type Definition

The leaf values of any request and input values to arguments are
Scalars (or Enums) and are defined with a name and a series of functions
used to parse input from ast or variables and to ensure validity.
File:           ~/.local/share/virtualenvs/gpug-9Pz75Evm/lib/python3.6/site-packages/graphene/types/scalars.py
Type:           SubclassWithMeta_Meta


In [39]:
graphene.List?

Init signature: graphene.List(of_type, *args, **kwargs)
Docstring:     
List Modifier

A list is a kind of type marker, a wrapping type which points to another
type. Lists are often created within the context of defining the fields of
an object type.
File:           ~/.local/share/virtualenvs/gpug-9Pz75Evm/lib/python3.6/site-packages/graphene/types/structures.py
Type:           type


In [26]:
graphene.Schema?

Init signature: graphene.Schema(query=None, mutation=None, subscription=None, directives=None, types=None, auto_camelcase=True)
Docstring:     
Schema Definition

A Schema is created by supplying the root types of each type of operation,
query and mutation (optional).
File:           ~/.local/share/virtualenvs/gpug-9Pz75Evm/lib/python3.6/site-packages/graphene/types/schema.py
Type:           type


## Hello world and introspection

- Define a graphene query
- Create a graphene schema
    - A schema requires a root query and optionally a root mutation and subscription

In [40]:
class Query(graphene.ObjectType):
    """Used in introspection."""
    hello = graphene.String(description="Used in instrospection.")

    def resolve_hello(self, info: graphene.ResolveInfo, **kwargs: typing.Dict) -> str:
        return "Hello world"

    
schema = graphene.Schema(query=Query, auto_camelcase=False)

### Introspection

Introspection is first class in GraphQL. Graphene builds documentation for your service using docstrings and attributes on your types and fields.

In [42]:
# GraphQL is introspectable by default
introspected = schema.introspect()
graphql_types = introspected['__schema']['types']
query_details = next(filter(lambda x: x['name'] == 'Query', graphql_types))

print('description:', query_details['description'])
print('field name:', query_details['fields'][0]['name'])
print('field description:', query_details['fields'][0]['description'])

description: Used in introspection.
field name: hello
field description: Used in instrospection.


### Excecution

In [43]:
result = schema.execute("""
    query {
        hello
    }
""")

print('\nExecution results:')
print('errors:', result.errors)
print('data:', dict(result.data))


Execution results:
errors: None
data: {'hello': 'Hello world'}


## More complex example

In [44]:
artists = [
    {'first_name': 'Ben', 'last_name': 'Howard'},
    {'first_name': 'Damien', 'last_name': 'Rice'},
    {'first_name': 'Noah', 'last_name': 'Gundersen'},
]


class ArtistType(graphene.ObjectType):
    """Artist type.

    An artist is descibed by their first and last name.
    """
    first_name = graphene.String()
    last_name = graphene.String()

    def resolve_first_name(self, info: graphene.ResolveInfo, **kwargs: typing.Dict) -> str:
        return self.first_name

    def resolve_last_name(self, info: graphene.ResolveInfo, **kwargs: typing.Dict) -> str:
        return self.last_name


class ArtistQuery(graphene.ObjectType):
    """Artist query.

    Exposes the artist data.
    """
    artists = graphene.List(ArtistType) # Note the list of artists

    def resolve_artists(self, info: graphene.ResolveInfo, **kwargs: typing.Dict) -> str:
        """Returns a list of artists"""
        return [ArtistType(**artist) for artist in artists]


class Query(ArtistQuery, graphene.ObjectType):
    pass


schema = graphene.Schema(query=Query, auto_camelcase=False)

### Execution

In [46]:
result = schema.execute("""
    query {
        artists {
            first_name
        }
    }
""")

result.data

OrderedDict([('artists',
              [OrderedDict([('first_name', 'Ben')]),
               OrderedDict([('first_name', 'Damien')]),
               OrderedDict([('first_name', 'Noah')])])])

In [50]:
result = schema.execute("""
    query {
        artists {
            last_name
        }
    }
""")

result.data

OrderedDict([('artists',
              [OrderedDict([('last_name', 'Howard')]),
               OrderedDict([('last_name', 'Rice')]),
               OrderedDict([('last_name', 'Gundersen')])])])

In [51]:
result = schema.execute("""
    query {
        artists {
            first_name
            last_name
        }
    }
""")

result.data

OrderedDict([('artists',
              [OrderedDict([('first_name', 'Ben'), ('last_name', 'Howard')]),
               OrderedDict([('first_name', 'Damien'), ('last_name', 'Rice')]),
               OrderedDict([('first_name', 'Noah'),
                            ('last_name', 'Gundersen')])])])

## Mutations

A mutation is a write followed by a fetch. However you can think of it as a way to send data to the graphql service. Therefore mutations are not limited to CRUD, they can execute any arbitrary function.

In [52]:
class CreateArtist(graphene.Mutation):
    """Mutation to create an artist."""
    class Arguments:
        first_name = graphene.String()
        last_name = graphene.String()

    status = graphene.String()
    artist = graphene.Field(lambda: ArtistType)

    def mutate(self, info: graphene.ResolveInfo, first_name: str, last_name: str):
        artists.append({
            'first_name': first_name,
            'last_name': last_name
        })
        
        return CreateArtist(
            artist=ArtistType(first_name=first_name, last_name=last_name), 
            status='OK'
        )


class SendEmail(graphene.Mutation):
    """Mutation to mock send an email."""
    class Arguments:
        name = graphene.String()

    status = graphene.String()
    
    def mutate(self, info: graphene.ResolveInfo, name: str):
        print('\nSending email:')
        print(f'Hello {name}, please send me your bank details.\n')

        return SendEmail(status="OK")


class Mutations(graphene.ObjectType):
    create_artist = CreateArtist.Field()
    send_email = SendEmail.Field()


schema = graphene.Schema(mutation=Mutations, auto_camelcase=False)

### Execution

In [58]:
print("Number of artists:", len(artists))

result = schema.execute("""
    mutation call_create_artist {
        create_artist(first_name: "John", last_name: "Lennon") {
            artist {
                first_name
                last_name
            }
            status
        }
    }
""")

print(result.data)
print("Number of artists:", len(artists))

Number of artists: 8
OrderedDict([('create_artist', OrderedDict([('artist', OrderedDict([('first_name', 'John'), ('last_name', 'Lennon')])), ('status', 'OK')]))])
Number of artists: 9


In [60]:
# Call the send email mutation
result = schema.execute("""
    mutation call_send_email {
        send_email(name: "Bradley") {
            status
        }
    }
""")

print("Send email mutation:", result.data)


Sending email:
Hello Bradley, please send me your bank details.

Send email mutation: OrderedDict([('send_email', OrderedDict([('status', 'OK')]))])
